# ДЗ Коллаборативная фильтрация

ПАКЕТ SURPRISE

используйте данные MovieLens 1M
можно использовать любые модели из пакета
получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [2]:
! pip install scikit-surprise

     |████████████████████████████████| 11.8MB 351kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670920 sha256=46caae234372e5abd2046772f80dda084fa1117d0c6a42cecdb780d0c9f6cb68
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [3]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import KNNWithZScore
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [7]:
# read and preprocess the movie data
movies = pd.read_table('movies.dat', sep='::', names=['movie_id', 'movie_name', 'tag'], engine='python')


# read the ratings data and merge it with movie data
ratings = pd.read_table("ratings.dat", sep="::",
                           names=["user_id", "movie_id", "rating", "timestamp"], engine='python')
movies_with_ratings = pd.merge(ratings, movies, on="movie_id")



In [11]:
ratings['user_id'].value_counts().head(5)

4169    2314
1680    1850
4277    1743
1941    1595
1181    1521
Name: user_id, dtype: int64

In [12]:
movies_with_ratings.dropna(inplace=True)

In [13]:
movies_with_ratings

,user_id,movie_id,rating,timestamp,movie_name,tag
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
1000204,5949,2198,5,958846401,Modulations (1998),Documentary
1000205,5675,2703,3,976029116,Broken Vessels (1998),Drama
1000206,5780,2845,1,958153068,White Boys (1999),Drama
1000207,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western


Датасет для surprise

In [16]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.user_id,
    'iid': movies_with_ratings.movie_name,
    'rating': movies_with_ratings.rating
})

In [17]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [18]:
trainset, testset = train_test_split(data, test_size=.15)

Item-based модель 

In [80]:
bsl_options = {'method': 'sgd',  'learning_rate': .0005, 'reg': .09 }
sim_options = {'name': 'pearson_baseline','user_based': False}  #'name': ['msd', 'cosine','pearson_baseline' pearson]

model_item = KNNBasic(k =10, bsl_options = bsl_options , sim_options = sim_options)
model_item.fit(trainset)
test_pred_i = model_item.test(testset)    
rez.append(['KNNBasic',  accuracy.rmse(test_pred_i),accuracy.mae(test_pred_i),bsl_options] )

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9164
MAE:  0.7134


User-based модель

In [39]:
rez = []

In [ ]:
bsl_options = {'method': 'als', 'n_epochs': 50}
sim_options = {'name': 'pearson_baseline','user_based': True}  #'name': ['msd', 'cosine','pearson_baseline' pearson]

model = KNNBasic(k =50, bsl_options = bsl_options , sim_options = sim_options)
model.fit(trainset)
test_pred = model.test(testset)    
rez.append(['KNNBasic',  accuracy.rmse(test_pred),accuracy.mae(test_pred),bsl_options] )

bsl_options = {'method': 'als', 'n_epochs': 50}
sim_options = {'name': 'cosine','user_based': True}  #'name': ['msd', 'cosine','pearson_baseline' pearson]
model = KNNBaseline(k =50, bsl_options = bsl_options , sim_options = sim_options)
model.fit(trainset)
test_pred = model.test(testset)    
rez.append(['KNNBaseline',   accuracy.rmse(test_pred),accuracy.mae(test_pred),bsl_options] )

bsl_options = {'method': 'sgd',  'learning_rate': .0005, 'reg': 1.5 }
test_pred = {}
model = KNNWithMeans(k =50, bsl_options = bsl_options , sim_options = sim_options)
model.fit(trainset)
test_pred = model.test(testset)    
rez.append(['KNNWithMeans',   accuracy.rmse(test_pred),accuracy.mae(test_pred),bsl_options] )

model = KNNBaseline(k =50, bsl_options = bsl_options , sim_options = sim_options)
model.fit(trainset)
test_pred = model.test(testset)    
rez.append(['KNNBaseline',   accuracy.rmse(test_pred),accuracy.mae(test_pred),bsl_options] )

model = KNNWithZScore(k =50, bsl_options = bsl_options , sim_options = sim_options)
model.fit(trainset)
test_pred = model.test(testset)    
rez.append(['KNNWithZScore',   accuracy.rmse(test_pred),accuracy.mae(test_pred),bsl_options] )

In [ ]:
bsl_options = {'method': 'als', 'n_epochs': 20, 'reg_u': .1,  'reg_i': 0.9 }  
sim_options = {'name': 'pearson_baseline'}

model = BaselineOnly(bsl_options = bsl_options )
model.fit(trainset)
test_pred_b = model.test(testset)    
rez.append(['BaselineOnly',  accuracy.rmse(test_pred_b),accuracy.mae(test_pred_b),bsl_options] )


bsl_options = {'method': 'sgd',  'learning_rate': .0005 }
model = BaselineOnly(bsl_options = bsl_options )
model.fit(trainset)
test_pred = model.test(testset)    
rez.append(['BaselineOnly',  accuracy.rmse(test_pred),accuracy.mae(test_pred),bsl_options] )

In [ ]:
bsl_options = 'n_factors = 50, n_epochs =20, biased = True '

model = NMF(n_factors = 50, n_epochs =20, biased = True )
model.fit(trainset)
test_pred = model.test(testset)    
rez.append(['NMF',  accuracy.rmse(test_pred),accuracy.mae(test_pred),bsl_options] )

In [ ]:
bsl_options = '-'

model = SlopeOne()
model.fit(trainset)
test_pred = model.test(testset)    
rez.append(['SlopeOne',  accuracy.rmse(test_pred),accuracy.mae(test_pred),bsl_options] )

In [ ]:
bsl_options = 'n_cltr_u = 100, n_cltr_i = 100 ,n_epochs =5,  random_state=42'

model = CoClustering(n_cltr_u = 100, n_cltr_i = 100 ,n_epochs =5,  random_state=42)
model.fit(trainset)
test_pred = model.test(testset)    
rez.append(['CoClustering',  accuracy.rmse(test_pred),accuracy.mae(test_pred),bsl_options] )


In [ ]:
bsl_options = '-'

model = NormalPredictor()
model.fit(trainset)
test_pred = model.test(testset)    
rez.append(['NormalPredictor',  accuracy.rmse(test_pred),accuracy.mae(test_pred),bsl_options] )

In [60]:
bsl_options = {'method': 'sgd',  'learning_rate': .005, 'reg': .09 }
sim_options = {'name': 'pearson_baseline','user_based': True}  #'name': ['msd', 'cosine','pearson_baseline' pearson]
model = KNNBaseline(k =50, bsl_options = bsl_options , sim_options = sim_options)
model.fit(trainset)
test_pred_knn = model.test(testset)   

rez.append(['KNNBaseline',   accuracy.rmse(test_pred_knn),accuracy.mae(test_pred_knn),bsl_options] )



Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8680
MAE:  0.6790


In [61]:
rez_tab = pd.DataFrame(rez)
rez_tab.columns = ['Model', 'RMSE', 'MAE', 'Params'] 

In [62]:
pd.set_option('max_colwidth', 100)
rez_tab.sort_values('RMSE')

,Model,RMSE,MAE,Params
17,KNNBaseline,0.867976,0.678954,"{'method': 'sgd', 'learning_rate': 0.005, 'reg': 0.09}"
3,KNNBaseline,0.884387,0.694571,"{'method': 'sgd', 'learning_rate': 0.0005, 'reg': 1.5}"
5,KNNBaseline,0.884387,0.694571,"{'method': 'sgd', 'learning_rate': 0.0005, 'reg': 1.5}"
6,KNNWithZScore,0.893203,0.690148,"{'method': 'sgd', 'learning_rate': 0.0005, 'reg': 1.5}"
4,KNNWithMeans,0.897171,0.695509,"{'method': 'sgd', 'learning_rate': 0.0005, 'reg': 1.5}"
1,KNNBaseline,0.897510,0.709236,"{'method': 'als', 'n_epochs': 50}"
15,SVD,0.902207,0.713755,"n_factors = 20, n_epochs =20, reg_pu = 0.1, reg_qi = 0.1, biased = True"
12,SlopeOne,0.906469,0.714120,-
11,SlopeOne,0.906469,0.714120,-
7,BaselineOnly,0.908023,0.716432,"{'method': 'als', 'n_epochs': 20, 'reg_u': 0.1, 'reg_i': 0.9}"


 0.867976  -  наилучший вариант  для модели KNNBaseline  с параметрами 'method': 'sgd', 'learning_rate': 0.005, 'reg': 0.09

Проверка предсказанной оценки на примерах:

In [72]:
model.predict(uid=472, iid='Silence of the Lambs, The (1991)')

Prediction(uid=472, iid='Silence of the Lambs, The (1991)', r_ui=None, est=4.804837615984431, details={'actual_k': 50, 'was_impossible': False})

In [66]:
movies_with_ratings.query('movie_name == \'Silence of the Lambs, The (1991)\' and user_id == 472')

,user_id,movie_id,rating,timestamp,movie_name,tag
136107,472,593,5,976222262,"Silence of the Lambs, The (1991)",Drama|Thriller


In [70]:
model.predict(uid=3, iid='Silence of the Lambs, The (1991)')

Prediction(uid=3, iid='Silence of the Lambs, The (1991)', r_ui=None, est=3.6687458351371522, details={'actual_k': 50, 'was_impossible': False})

In [71]:
movies_with_ratings.query('movie_name == \'Silence of the Lambs, The (1991)\' and user_id == 3')

,user_id,movie_id,rating,timestamp,movie_name,tag
135922,3,593,3,978297018,"Silence of the Lambs, The (1991)",Drama|Thriller


Для сравнения item-based и user-based моделей:

In [82]:
print("Оценка user-based:", model.predict(uid=2, iid='Manhattan (1979)').est )   

Оценка user-based: 3.478024836487156


In [81]:
print("Оценка item-based:", model_item.predict(uid=2, iid='Manhattan (1979)').est)

Оценка item-based: 3.238521187752126


In [76]:
movies_with_ratings.query('movie_name == \'Manhattan (1979)\' and user_id == 2')

,user_id,movie_id,rating,timestamp,movie_name,tag
177108,2,1244,3,978299143,Manhattan (1979),Comedy|Drama|Romance


In [83]:
print("Оценка user-based:", model.predict(uid=2, iid='Driving Miss Daisy (1989)').est)

Оценка user-based: 4.425529309536123


In [84]:
print("Оценка item-based:",model_item.predict(uid=2, iid='Driving Miss Daisy (1989)').est)

Оценка item-based: 4.746453887907432


In [75]:
movies_with_ratings.query('movie_name == \'Driving Miss Daisy (1989)\' and user_id == 2')

,user_id,movie_id,rating,timestamp,movie_name,tag
45034,2,1962,5,978298813,Driving Miss Daisy (1989),Drama
